In [33]:
from google.colab import drive
drive.mount('/content/gdrive')
# 경로 변경
%cd /content/gdrive/MyDrive/pytest_img/opencv/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/pytest_img/opencv


In [34]:
import cv2
import numpy as np
import dlib
import os

seconds = 1 # 동영상을 읽을 시간을 초 단위로 설정

video_path = '/content/gdrive/MyDrive/pytest_img/opencv/eye_opening.mp4' # video path
output_dir = '/content/gdrive/MyDrive/pytest_img/opencv/results' # 결과를 저장할 폴더

In [35]:
# 얼굴 영역 인식 함수
detector = dlib.get_frontal_face_detector()

# 얼굴의 68개 지점 인식 모델 로드
sp =dlib.shape_predictor("/content/gdrive/MyDrive/pytest_img/dlib/shape_predictor_68_face_landmarks.dat")

In [36]:
RIGHT_EYE = list(range(36, 42)) # 오른쪽 눈. 36~41까지 생성
LEFT_EYE = list(range(42, 48)) # 왼쪽 눈. 42~47까지 생성

def eye_aspect_ratio(eye_points):
  # 눈의 세로길이
  A = np.linalg.norm(eye_points[1] - eye_points[5]) # 벡터의 길이 계산
  B = np.linalg.norm(eye_points[2] - eye_points[4])
  # 눈의 가로 길이
  C = np.linalg.norm(eye_points[0] - eye_points[3])

  EAR = (A + B) / (2.0 * C)
  return EAR

In [37]:
def predict(frame, predictor, LEFT_EYE, RIGHT_EYE):
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 잘 인식하도록 그레이스케일로 변환
  dets = detector(gray, 1) # 얼굴 감지
  for det in dets:
    sp = predictor(gray, det)
    points = np.array([[p.x, p.y] for p in sp.parts()])
    left_eye_EAR = eye_aspect_ratio(points[LEFT_EYE]) # 왼쪽 눈 EAR
    right_eye_EAR = eye_aspect_ratio(points[RIGHT_EYE]) # 오른쪽 눈 EAR

    # 눈 위치에 EAR 값 표시 (왼쪽 눈, 오른쪽 눈)
    cv2.putText(frame, f"{left_eye_EAR:.2f}", (points[LEFT_EYE[0]][0], points[LEFT_EYE[0]][1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3) #cv2.FONT_HERSHEY_SIMPLEX는 글
    cv2.putText(frame, f"{right_eye_EAR:.2f}", (points[RIGHT_EYE[0]][0], points[RIGHT_EYE[0]][1]+ 20), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

    return frame

In [38]:
capture = cv2.VideoCapture(video_path) # 비디오 파일 로드

# 비디오가 마지막 프레임에 도달한 경우, 처음부터 다시 시작하기
# CAP_PROP_POS_FRAMES는 비디오의 현재 프레임, CAP_PROP_FRAME_COUNT은 비디오의 총 프레임 수
if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
    capture.set(cv2.CAP_PROP_POS_FRAMES, 0) # 첫 번째 프레임으로 변경

fps = capture.get(cv2.CAP_PROP_FPS) # 초당 프레임 수 얻기
total_frames = int(fps * seconds) # 현재 seconds == 1
print('total 프레임:', total_frames) # 1초인 경우, 25 프레임
# 출력 디렉토리 생성
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

total 프레임: 25


In [43]:
# fps = capture.get(cv2.CAP_PROP_FPS) # 초당 프레임 수 얻기
# total_frame_count = capture.get(cv2.CAP_PROP_FRAME_COUNT) # 전체 프레임 수 얻기
# seconds = total_frame_count/fps # 동영상 재생 시간 얻는 코드

7.68

In [23]:
frame_count = 0
while frame_count < total_frames:
  ok, frame = capture.read()
  if not ok:
    print("프레임 읽기에 실패.")
    break
  frame = predict(frame,sp,LEFT_EYE,RIGHT_EYE)

  output_path = os.path.join(output_dir,f'frame_{frame_count:04d}.jpg')
  if not os.path.exists(output_path):
    cv2.imwrite(output_path,frame)
  print("Saved to:",output_path)
  frame_count += 1

Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results/frame_0000.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results/frame_0001.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results/frame_0002.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results/frame_0003.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results/frame_0004.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results/frame_0005.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results/frame_0006.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results/frame_0007.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results/frame_0008.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results/frame_0009.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results/frame_0010.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results/frame_0011.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results/frame_0012.jpg
Saved to: /content/gdrive

In [24]:
seconds = 1 # 동영상을 읽을 시간을 초 단위로 설정

# 비디오 경로 설정 및 결과 저장 경로 설정
video_path = '/content/gdrive/MyDrive/pytest_img/opencv/eye_opening.mp4'
output_dir = '/content/gdrive/MyDrive/pytest_img/opencv/results2'

In [25]:
# 나이 구간 정의
age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']

# 성별 구분 정의
gender_list = ["Male", 'Female']

In [26]:
# 얼굴 영역 인식 함수
detector = dlib.get_frontal_face_detector()

# 나이 인식 함수
age_detector = cv2.dnn.readNetFromCaffe("/content/gdrive/MyDrive/pytest_img/dlib/deploy_age.prototxt", "/content/gdrive/MyDrive/pytest_img/dlib/age_net.caffemodel")

# 성별 인식 함수
gender_detector = cv2.dnn.readNetFromCaffe("/content/gdrive/MyDrive/pytest_img/dlib/deploy_gender.prototxt", "/content/gdrive/MyDrive/pytest_img/dlib/gender_net.caffemodel")

In [27]:
# 비디오 파일 로드
capture = cv2.VideoCapture(video_path)

# 비디오가 마지막 프레임에 도달한 경우, 처음부터 다시 시작하기
if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
  capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

fps = capture.get(cv2.CAP_PROP_FPS) # 초당 프레임 수 얻기
total_frames = int(fps * seconds)
print('total 프레임:', total_frames)

# 출력 디렉토리 생성
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

total 프레임: 25


In [28]:
def predict(frame, age_detector, gender_detector, age_list, gender_list):
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 잘 인식하도록 그레이스케일로 변환
  dets = detector(gray, 1) # 얼굴 감지
  for det in dets:
    x1, y1, x2, y2 = det.left(), det.top(), det.right(), det.bottom()
    face_img = frame[y1:y2, x1:x2].copy() # 원본 이미지에서 얼굴 영역만 copy
    blob = cv2.dnn.blobFromImage(face_img, scalefactor=1, size=(227, 227),mean=(78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    age_detector.setInput(blob) # 나이 예측 모델의 입력으로 설정
    age_preds = age_detector.forward() # 모델을 실행하여 예측 결과를 얻음
    age = age_list[age_preds[0].argmax()] # 가장 높은 확률을 선택
    gender_detector.setInput(blob) # 성별 예측
    gender_preds = gender_detector.forward()
    gender = gender_list[gender_preds[0].argmax()]
    # 나이와 성별 표시
    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)
    text = f'{gender}, {age}'
    cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 10) # 검정색 배경
    cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3) # 녹색 텍스트
  return frame

In [29]:
fourcc = cv2.VideoWriter_fourcc(*'XVID') # 비디오 표준 코딩을 사용하는 XVID를 사용하여 비디오 생성

# 초당 25 프레임 사용
# 원본 비디오 프레임의 너비(capture.get(3))와 높이(capture.get(4))를 반환하여 같은 크기로 비디오 생성
out = cv2.VideoWriter(output_dir+'/output.avi', fourcc, 25.0, (int(capture.get(3)), int(capture.get(4))))

In [30]:
frame_count = 0
while frame_count < total_frames:
  ok, frame = capture.read()
  if not ok:
    print("프레임 읽기에 실패했습니다. 종료.")
    break
  frame = predict(frame, age_detector, gender_detector, age_list, gender_list) # 예측 수행
  out.write(frame) # 처리된 프레임을 동영상 파일로 저장

  output_path = os.path.join(output_dir, f'frame_{frame_count:04d}.jpg') # 이미지 파일도 저장
  cv2.imwrite(output_path, frame)
  print("Saved to:", output_path)
  frame_count += 1

  # 사용자가 프로세스 중간에 종료할 수 있도록 키 입력 대기 및 종료 처리
  key = cv2.waitKey(10)
  if key == ord('q'):
    print("사용자가 종료를 요청했습니다.")
    break

out.release()
capture.release()
cv2.destroyAllWindows()

Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results2/frame_0000.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results2/frame_0001.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results2/frame_0002.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results2/frame_0003.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results2/frame_0004.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results2/frame_0005.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results2/frame_0006.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results2/frame_0007.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results2/frame_0008.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results2/frame_0009.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results2/frame_0010.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results2/frame_0011.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/opencv/results2/frame_0012.jpg
Saved to: /c